In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, initcap, trim, lower, upper, regexp_replace, regexp_extract,
    when, to_date, lit, current_date, datediff, length, concat_ws
)
from datetime import datetime, timedelta

In [0]:
df_raw = spark.table("prod.bronze_raw.transactions")

##### date handling

- Convert `Transaction_Date`, `DOB`, `Expiry_Date` to DATE
- Filter out records:
  - `DOB` must be in the past: `datediff(current_date(), DOB) > 0`
  - `Expiry_Date` must be in the future: `datediff(Expiry_Date, current_date()) > 0`

In [0]:
df = df_raw.withColumn("DOB", to_date(col("DOB"), "yyyy-MM-dd")) \
           .withColumn("Transaction_Date", to_date(col("Transaction_Date"), "yyyy-MM-dd")) \
           .withColumn("Expiry_Date", to_date(col("Expiry_Date"), "yyyy-MM-dd")) \
           .filter((datediff(current_date(), col("DOB")) > 0) &
                   (datediff(col("Expiry_Date"), current_date()) > 0))

- For fields like `Customer_Name`, `Merchant_Name`, `City`, `State`, `Country`, `Issuer_Bank`:
  - Remove non-alphabetic characters
  - Trim whitespace
  - Apply InitCap() (title case)
- Remove rows with missing values in the columns: `Transaction_ID`, `Customer_ID`, and `Card_ID`
- Remove duplicate rows based on the `Transaction_ID` column, keeping the first occurrence

In [0]:
text_fields = [
    "Customer_Name", "Merchant_Name", "Customer_City", "City", "State",
    "Country", "Merchant_Country", "Issuer_Bank", "Card_Type"
]

for f in text_fields:
    df = df.withColumn(f,
            initcap(trim(regexp_replace(col(f), r"[^a-zA-Z\\s]", ""))))

df.display()

Transaction_ID Customer_ID Card_ID Merchant_ID Location_ID Transaction_Date DOB Expiry_Date Customer_Name Gender Email Phone Customer_City Card_Type Issuer_Bank Card_Tier Transaction_Status Transaction_Type Transaction_Amount Merchant_Name Merchant_Category Merchant_Country City State Country year T1000 C1000 Card1000 M1000 L1000 2024-10-30 1952-01-19 2025-09-07 Danielletaylor M null 7117550026 Ahmedabad Debit Rhodesplc Silver FAIL pos 71630.36 Garzainc Grocery Inda Lucknow Telangana India 2024 T1005 C1005 Card1005 M1005 L1005 2024-11-23 1962-08-22 2026-01-29 Carolyndaniel F carolyn.daniel@gmail.com 9895169156 Lucknow Debit Fordllc Platinum Failed onlne 92898.46 Wattsrobinsonandnguyen Grocery India Mumbai Telangana India 2024 T1007 C1007 Card1007 M1007 L1007 2024-04-24 2004-09-03 2029-05-17 Charlesmcgee O charles.mcgee@rediffmail.com 6634806607 Ahmedabad Credit Bakerrichardsandhurst Gold Failed POS 11015.98 Coxosborn Retail India Chandigarh Maharashtra India 2024 T1009 C1009 Card1009 M1009 L1009 2024-08-05 1961-07-31 2030-03-06 Alexandrale F alexandra.le@yahoo.com 6800849550 Kolkata Credit Dyerpotterandmack Gold FAIL atm 65771.58 Rodriguezgraham Electronics India Kolkata Gujarat India 2024 T1010 C1010 Card1010 M1010 L1010 2024-06-08 1944-09-10 2027-09-18 Pamelaromero O pamela.romero@gmail.com 6022988318 Chandigarh Debit Harrellllc Silver FAIL Online 61206.61 Romerogonzalezandbrooks Grocery India Mumbai Maharashtra Inda 2024 T1014 C1014 Card1014 M1014 L1014 2024-03-10 1945-11-26 2030-03-08 Danieljones F daniel.jones@yahoo.com 9095776306 Chandigarh Debit Brownjonesandjohnson Gold Success onlne 31460.17 Jonesyoung Travel Inda Chennai Delhi India 2024 T1015 C1015 Card1015 M1015 L1015 2024-11-22 1977-07-04 2027-04-18 Mirandakhan M miranda.khan@yahoo.com 8483276346 Kolkata Debit Kennedymuellerandaguirre Silver Failed onlne 79394.17 Davisltd Grocery India Chennai Karnataka India 2024 T1016 C1016 Card1016 M1016 L1016 2024-05-21 1968-08-01 2029-01-28 Scottbrown M scott.brown@gmail.com 7733168887 Chandigarh Credit Joycegarciaandpowell Silver FAIL pos 80430.57 Mcbriderodriguez Electronics Inda Chandigarh Punjab Inda 2024 T1022 C1022 Card1022 M1022 L1022 2024-09-06 1961-03-24 2030-05-25 Lisabarnes M lisa.barnes@gmail.com 9246308127 Chandigarh Credit Cunninghamstewartandgreen Platinum Fraudulent atm 88043.97 Moranallenandfoster Grocery Inda Ahmedabad Gujarat India 2024 T1023 C1023 Card1023 M1023 L1023 2024-11-24 1988-05-26 2028-01-10 Timothywalls M timothy.walls@yahoo.com 7354547372 Chandigarh Credit Ellisplc Platinum Failed ATM 12153.52 Jonesandsons Travel India Delhi Westbengal India 2024 T1024 C1024 Card1024 M1024 L1024 2024-07-18 1962-07-09 2026-08-07 Shirleysuarez M shirley.suarez@yahoo.com 6561848544 Lucknow Debit Mitchellkim Platinum sucsess Online 5232.53 Warnercarrandarcher Grocery India Kochi Westbengal India 2024 T1026 C1026 Card1026 M1026 L1026 2024-08-24 1955-01-19 2027-10-16 Danielcooper M daniel.cooper@outlook.com 9022758112 Chennai Debit Waltersbakerandfreeman Platinum Failed Online 26636.68 Davisgroup Electronics India Ahmedabad Westbengal Inda 2024 T1034 C1034 Card1034 M1034 L1034 2024-03-16 1994-01-19 2028-02-21 Williamwilson O william.wilson@rediffmail.com 6255929625 Ahmedabad Debit Leewilliamsandgraham Silver Success atm 74492.42 Frostryanandball Travel India Delhi Kerala India 2024 T1036 C1036 Card1036 M1036 L1036 2024-07-19 1965-07-09 2028-12-11 Michaelspencer O michael.spencer@yahoo.com 9983705466 Delhi Debit Fishergroup null Fraudulent Online 82259.49 Adkinsthompsonandcarroll Travel Inda Delhi Maharashtra Inda 2024 T1041 C1041 Card1041 M1041 L1041 2024-11-21 1980-08-13 2030-03-24 Tiffanyvance F tiffany.vance@yahoo.com 7483733296 Mumbai Debit Briggsmcclainandsimmons Platinum Success Online 9213.36 Griffinmooreandhensley Travel Inda Mumbai Uttarpradesh India 2024 T1042 C1042 Card1042 M1042 L1042 2024-06-05 1962-02-10 2027-07-30 Destinystrickland F destiny.strickland@rediffmail.com 6963526218 Chandigarh Credit Simshil

- Clean `Gender`:
  - Normalize variants (e.g., "M", "MALE" → "MALE"; "F", "FEMALE" → "FEMALE")
  - Trim whitespace

In [0]:
from pyspark.sql.functions import col, upper, trim, when, lit

df = df.withColumn(
    "Gender",
    when(upper(trim(col("Gender"))) == "M", upper(trim(lit("Male"))))
    .when(upper(trim(col("Gender"))) == "F", upper(trim(lit("Female"))))
    .otherwise(upper(trim(lit("Other"))))
)


Transaction_ID Customer_ID Card_ID Merchant_ID Location_ID Transaction_Date DOB Expiry_Date Customer_Name Gender Email Phone Customer_City Card_Type Issuer_Bank Card_Tier Transaction_Status Transaction_Type Transaction_Amount Merchant_Name Merchant_Category Merchant_Country City State Country year T1000 C1000 Card1000 M1000 L1000 2024-10-30 1952-01-19 2025-09-07 Danielletaylor MALE null 7117550026 Ahmedabad Debit Rhodesplc Silver FAIL pos 71630.36 Garzainc Grocery Inda Lucknow Telangana India 2024 T1005 C1005 Card1005 M1005 L1005 2024-11-23 1962-08-22 2026-01-29 Carolyndaniel FEMALE carolyn.daniel@gmail.com 9895169156 Lucknow Debit Fordllc Platinum Failed onlne 92898.46 Wattsrobinsonandnguyen Grocery India Mumbai Telangana India 2024 T1007 C1007 Card1007 M1007 L1007 2024-04-24 2004-09-03 2029-05-17 Charlesmcgee OTHER charles.mcgee@rediffmail.com 6634806607 Ahmedabad Credit Bakerrichardsandhurst Gold Failed POS 11015.98 Coxosborn Retail India Chandigarh Maharashtra India 2024 T1009 C1009 Card1009 M1009 L1009 2024-08-05 1961-07-31 2030-03-06 Alexandrale FEMALE alexandra.le@yahoo.com 6800849550 Kolkata Credit Dyerpotterandmack Gold FAIL atm 65771.58 Rodriguezgraham Electronics India Kolkata Gujarat India 2024 T1010 C1010 Card1010 M1010 L1010 2024-06-08 1944-09-10 2027-09-18 Pamelaromero OTHER pamela.romero@gmail.com 6022988318 Chandigarh Debit Harrellllc Silver FAIL Online 61206.61 Romerogonzalezandbrooks Grocery India Mumbai Maharashtra Inda 2024 T1014 C1014 Card1014 M1014 L1014 2024-03-10 1945-11-26 2030-03-08 Danieljones FEMALE daniel.jones@yahoo.com 9095776306 Chandigarh Debit Brownjonesandjohnson Gold Success onlne 31460.17 Jonesyoung Travel Inda Chennai Delhi India 2024 T1015 C1015 Card1015 M1015 L1015 2024-11-22 1977-07-04 2027-04-18 Mirandakhan MALE miranda.khan@yahoo.com 8483276346 Kolkata Debit Kennedymuellerandaguirre Silver Failed onlne 79394.17 Davisltd Grocery India Chennai Karnataka India 2024 T1016 C1016 Card1016 M1016 L1016 2024-05-21 1968-08-01 2029-01-28 Scottbrown MALE scott.brown@gmail.com 7733168887 Chandigarh Credit Joycegarciaandpowell Silver FAIL pos 80430.57 Mcbriderodriguez Electronics Inda Chandigarh Punjab Inda 2024 T1022 C1022 Card1022 M1022 L1022 2024-09-06 1961-03-24 2030-05-25 Lisabarnes MALE lisa.barnes@gmail.com 9246308127 Chandigarh Credit Cunninghamstewartandgreen Platinum Fraudulent atm 88043.97 Moranallenandfoster Grocery Inda Ahmedabad Gujarat India 2024 T1023 C1023 Card1023 M1023 L1023 2024-11-24 1988-05-26 2028-01-10 Timothywalls MALE timothy.walls@yahoo.com 7354547372 Chandigarh Credit Ellisplc Platinum Failed ATM 12153.52 Jonesandsons Travel India Delhi Westbengal India 2024 T1024 C1024 Card1024 M1024 L1024 2024-07-18 1962-07-09 2026-08-07 Shirleysuarez MALE shirley.suarez@yahoo.com 6561848544 Lucknow Debit Mitchellkim Platinum sucsess Online 5232.53 Warnercarrandarcher Grocery India Kochi Westbengal India 2024 T1026 C1026 Card1026 M1026 L1026 2024-08-24 1955-01-19 2027-10-16 Danielcooper MALE daniel.cooper@outlook.com 9022758112 Chennai Debit Waltersbakerandfreeman Platinum Failed Online 26636.68 Davisgroup Electronics India Ahmedabad Westbengal Inda 2024 T1034 C1034 Card1034 M1034 L1034 2024-03-16 1994-01-19 2028-02-21 Williamwilson OTHER william.wilson@rediffmail.com 6255929625 Ahmedabad Debit Leewilliamsandgraham Silver Success atm 74492.42 Frostryanandball Travel India Delhi Kerala India 2024 T1036 C1036 Card1036 M1036 L1036 2024-07-19 1965-07-09 2028-12-11 Michaelspencer OTHER michael.spencer@yahoo.com 9983705466 Delhi Debit Fishergroup null Fraudulent Online 82259.49 Adkinsthompsonandcarroll Travel Inda Delhi Maharashtra Inda 2024 T1041 C1041 Card1041 M1041 L1041 2024-11-21 1980-08-13 2030-03-24 Tiffanyvance FEMALE tiffany.vance@yahoo.com 7483733296 Mumbai Debit Briggsmcclainandsimmons Platinum Success Online 9213.36 Griffinmooreandhensley Travel Inda Mumbai Uttarpradesh India 2024 T1042 C1042 Card1042 M1042 L1042 2024-06-05 1962-02-10 2027-07-30 Destinystrickland FEMALE destiny.

In [0]:
df = df.dropna(subset=["Transaction_ID", "Customer_ID", "Card_ID"]) \
           .drop_duplicates(["Transaction_ID"])

- Clean `Card_Tier`:
  - Trim whitespace
  - Convert to UPPERCASE

In [0]:
df = df.withColumn("Card_Tier", upper(trim(col("Card_Tier"))))

- Clean `Email`:
  - Regex match: `^[\w\.-]+@[\w\.-]+\.\w+$`
    - If valid → lowercased, trimmed
    - If invalid → set to NULL

In [0]:
df = df.withColumn(
        "Email",
        when(col("Email").rlike(r"^[\w\.-]+@[\w\.-]+\.\w+$"), lower(trim(col("Email"))))
        .otherwise(lit(None)))


- If `Customer_Name` is NULL and `Email` is valid:
  - Extract string before `@` and use as name
  - Else retain original value

In [0]:
df = df.withColumn(
    "Customer_Name",
    when(col("Customer_Name").isNull(),
         regexp_extract(col("Email"), r"^([^@]+)", 1))
    .otherwise(col("Customer_Name"))
)

- Clean `Phone_Number`:
  - Keep only digits
  - Must be exactly 10 digits
    - If valid → retain
    - If invalid → set to NULL

In [0]:
df = df.withColumn(
    "Phone",
    when((col("Phone").rlike(r"^\d{10}$")), col("Phone"))
    .otherwise(lit(None))
)

# df2.display()

- Replace the following placeholder values with NULL:
  - “-”, “null”, “unknown”, “N/A”, “na”, “none”, “None”
- Applicable fields:
  - Customer_Name, Merchant_Name, City, State, Country, Bank, Card_Tier, Transaction_Status, Transaction_Type, Email, Phone

In [0]:
placeholders = ["-", "null", "unknown", "N/A", "na", "none", "None"]
string_cols = [
    "Customer_Name", "Merchant_Name", "Customer_City", "City", "State",
    "Country", "Merchant_Country", "Issuer_Bank", "Card_Type",
    "Transaction_Status", "Transaction_Type", "Email", "Phone"
]

for f in string_cols:
    df = df.withColumn(
            f,
            when(col(f).isin(placeholders), lit(None)).otherwise(col(f))
        )

- Correct known typos:
  - "Tmil Nadu" → "Tamil Nadu" in State
  - "Inda" → "India" in Country

In [0]:
df = (df
      .withColumn("State",   regexp_replace(col("State"),   "Tmil Nadu", "Tamil Nadu"))
      .withColumn("Country", regexp_replace(col("Country"), "Inda",      "India")))

- Add `Is_Valid_Record` column:
  - True if all conditions are met:
    - `Transaction_Amount` > 0
    - `DOB` < current_date()
    - `Expiry_Date` > current_date()
  - Otherwise, False

In [0]:


df = df.withColumn(
    "Is_Valid_Record",
    when(
        (col("Transaction_Amount") > 0) &
        (datediff(current_date(), col("DOB")) > 0) &               # DOB must be in the past
        (datediff(col("Expiry_Date"), current_date()) > 0),  # Expiry date must be in the future
        lit(True)
    ).otherwise(lit(False))
)


In [0]:
df.display()

Transaction_ID Customer_ID Card_ID Merchant_ID Location_ID Transaction_Date DOB Expiry_Date Customer_Name Gender Email Phone Customer_City Card_Type Issuer_Bank Card_Tier Transaction_Status Transaction_Type Transaction_Amount Merchant_Name Merchant_Category Merchant_Country City State Country year Is_Valid_Record T1000 C1000 Card1000 M1000 L1000 2024-10-30 1952-01-19 2025-09-07 Danielletaylor MALE null 7117550026 Ahmedabad Debit Rhodesplc SILVER FAIL pos 71630.36 Garzainc Grocery Inda Lucknow Telangana India 2024 true T1001 C1001 Card1001 M1001 L1001 2023-07-15 1950-11-06 2027-05-02 Lancehoffman MALE lance.hoffman@outlook.com 9086875935 Mumbai Credit Santosgardnerandrobinson GOLD Fraudulent atm 61890.12 Ramirezboothandblake Grocery India Lucknow Westbengal India 2023 true T1002 C1002 Card1002 M1002 L1002 2023-10-24 1991-12-25 2029-01-25 Christopherbernard MALE christopher.bernard@outlook.com 8760652429 Bangalore Debit Garciajames SILVER Fraudulent Online 22039.56 Abbottmunoz Travel India Ahmedabad Uttarpradesh India 2023 true T1003 C1003 Card1003 M1003 L1003 2025-03-09 1947-06-06 2029-05-19 Valeriegray MALE valerie.gray@yahoo.com 8745391478 Lucknow Debit Carterfullerandmcclure GOLD sucsess atm 16082.2 Raybush Retail Inda Mumbai Punjab India 2025 true T1004 C1004 Card1004 M1004 L1004 2023-07-27 1989-06-21 2027-05-31 Victoriawyatt OTHER victoria.wyatt@outlook.com 9077975168 Chandigarh Credit Jamesgroup GOLD Fraudulent POS 19971.22 Flowersmartinandkelly null India Kolkata Delhi India 2023 true T1005 C1005 Card1005 M1005 L1005 2024-11-23 1962-08-22 2026-01-29 Carolyndaniel FEMALE carolyn.daniel@gmail.com 9895169156 Lucknow Debit Fordllc PLATINUM Failed onlne 92898.46 Wattsrobinsonandnguyen Grocery India Mumbai Telangana India 2024 true T1006 C1006 Card1006 M1006 L1006 2025-02-07 1997-04-12 2025-12-12 Andrewstewart FEMALE andrew.stewart@yahoo.com null Mumbai Credit Jonesgentry GOLD Success Online 5981.49 Walteredwardsandrios Grocery Inda Kochi Kerala India 2025 true T1007 C1007 Card1007 M1007 L1007 2024-04-24 2004-09-03 2029-05-17 Charlesmcgee OTHER charles.mcgee@rediffmail.com 6634806607 Ahmedabad Credit Bakerrichardsandhurst GOLD Failed POS 11015.98 Coxosborn Retail India Chandigarh Maharashtra India 2024 true T1008 C1008 Card1008 M1008 L1008 2023-10-07 1979-02-23 2029-09-02 Anthonyrodriguez OTHER anthony.rodriguez@gmail.com 7142403927 Ahmedabad Credit Moorebeckerandcarlson SILVER FAIL onlne 42358.45 Jamesferrell Retail Inda Hyderabad Karnataka India 2023 true T1009 C1009 Card1009 M1009 L1009 2024-08-05 1961-07-31 2030-03-06 Alexandrale FEMALE alexandra.le@yahoo.com 6800849550 Kolkata Credit Dyerpotterandmack GOLD FAIL atm 65771.58 Rodriguezgraham Electronics India Kolkata Gujarat India 2024 true T1010 C1010 Card1010 M1010 L1010 2024-06-08 1944-09-10 2027-09-18 Pamelaromero OTHER pamela.romero@gmail.com 6022988318 Chandigarh Debit Harrellllc SILVER FAIL Online 61206.61 Romerogonzalezandbrooks Grocery India Mumbai Maharashtra India 2024 true T1011 C1011 Card1011 M1011 L1011 2023-10-25 1981-03-06 2029-12-25 Lindaburns OTHER linda.burns@yahoo.com 7685290680 Ahmedabad Debit Hickmanllc SILVER Fraudulent POS 13168.88 Brownplc Grocery Inda Bangalore Telangana India 2023 true T1012 C1012 Card1012 M1012 L1012 2023-08-22 1950-04-20 2029-08-09 Vanessapatel MALE vanessa.patel@gmail.com 6822178461 Chennai Debit Andersongroup GOLD Failed pos 86474.11 Martinroseandobrien Grocery India Chennai Maharashtra India 2023 true T1013 C1013 Card1013 M1013 L1013 2023-12-03 1981-03-04 2027-03-08 Jenniferpowers MALE jennifer.powers@yahoo.com 9388263207 Mumbai Debit Kimholmes GOLD FAIL ATM 50989.63 Wrightinc Electronics India Hyderabad Uttarpradesh India 2023 true T1014 C1014 Card1014 M1014 L1014 2024-03-10 1945-11-26 2030-03-08 Danieljones FEMALE daniel.jones@yahoo.com 9095776306 Chandigarh Debit Brownjonesandjohnson GOLD Success onlne 31460.17 Jonesyoung Travel Inda Chennai Delhi India 2024 true T1015 C1015 Card1015 M1015 L1015 2024-11-22 1977-07-04 2027-

- Add `Invalid_Reason` column with priority:
  - If `Transaction_Amount` <= 0 → "Invalid Transaction Amount"
  - Else if `DOB` >= current_date() → "Future DOB"
  - Else if `Expiry_Date` <= current_date() → "Card Expired"
  - Else NULL

In [0]:
# 2. Add Invalid_Reason
df = df.withColumn(
    "Invalid_Reason",
    when(col("Transaction_Amount") <= 0, lit("Invalid Transaction Amount"))
    .when(datediff(current_date(), col("DOB")) <= 0, lit("Future DOB"))
    .when(datediff(col("Expiry_Date"), current_date()) <= 0, lit("Card Expired"))
    .otherwise(lit(None))
)

city_state_map = {
    "Bangalore"  : ("Karnataka",       "L1001"),
    "Mumbai"     : ("Maharashtra",     "L1002"),
    "Chennai"    : ("Tamil Nadu",      "L1003"),
    "Delhi"      : ("Delhi",           "L1004"),
    "Kolkata"    : ("West Bengal",     "L1005"),
    "Hyderabad"  : ("Telangana",       "L1006"),
    "Lucknow"    : ("Uttar Pradesh",   "L1007"),
    "Ahmedabad"  : ("Gujarat",         "L1008"),
    "Chandigarh" : ("Punjab",          "L1009"),
    "Kochi"      : ("Kerala",          "L1010"),
}
- Normalize City and State to title case (trimmed)
- Use reference mapping (`city_state_map`) to:
  - Force correct State for known cities
  - Enrich with `Location_ID`
- Default `Location_ID` = 'L9999' if no match found

In [0]:
from pyspark.sql.functions import col, when, lit, trim, initcap

def apply_city_state(df, mapping):
    df_out = df
    for city, (state, loc) in mapping.items():
        df_out = df_out.withColumn(
            "State",
            when(col("City") == city, lit(state)).otherwise(col("State"))
        ).withColumn(
            "Location_ID",
            when((col("City") == city) & (col("State") == state), lit(loc)).otherwise(col("Location_ID"))
        )
    df_out = df_out.fillna({"Location_ID": "L9999"})
    return df_out

# Example usage and explanation
city_state_map = {
    "Bangalore"  : ("Karnataka",       "L1001"),
    "Mumbai"     : ("Maharashtra",     "L1002"),
    "Chennai"    : ("Tamil Nadu",      "L1003"),
    "Delhi"      : ("Delhi",           "L1004"),
    "Kolkata"    : ("West Bengal",     "L1005"),
    "Hyderabad"  : ("Telangana",       "L1006"),
    "Lucknow"    : ("Uttar Pradesh",   "L1007"),
    "Ahmedabad"  : ("Gujarat",         "L1008"),
    "Chandigarh" : ("Punjab",          "L1009"),
    "Kochi"      : ("Kerala",          "L1010"),
}

# Suppose df is already defined and cleaned as in previous cells
df_result = apply_city_state(df, city_state_map)

# Show the result to verify the function works as expected
# display(df_result)

# Explain: The function iterates over the mapping, updating 'State' and 'Location_ID' columns based on city matches.
# If a city matches, its state is set; if both city and state match, the location_id is set.
# Any missing location_id is filled with 'L9999'.

Transaction_ID Customer_ID Card_ID Merchant_ID Location_ID Transaction_Date DOB Expiry_Date Customer_Name Gender Email Phone Customer_City Card_Type Issuer_Bank Card_Tier Transaction_Status Transaction_Type Transaction_Amount Merchant_Name Merchant_Category Merchant_Country City State Country year Is_Valid_Record Invalid_Reason T1000 C1000 Card1000 M1000 L1007 2024-10-30 1952-01-19 2025-09-07 Danielletaylor MALE null 7117550026 Ahmedabad Debit Rhodesplc SILVER FAIL pos 71630.36 Garzainc Grocery Inda Lucknow Uttar Pradesh India 2024 true null T1001 C1001 Card1001 M1001 L1007 2023-07-15 1950-11-06 2027-05-02 Lancehoffman MALE lance.hoffman@outlook.com 9086875935 Mumbai Credit Santosgardnerandrobinson GOLD Fraudulent atm 61890.12 Ramirezboothandblake Grocery India Lucknow Uttar Pradesh India 2023 true null T1002 C1002 Card1002 M1002 L1008 2023-10-24 1991-12-25 2029-01-25 Christopherbernard MALE christopher.bernard@outlook.com 8760652429 Bangalore Debit Garciajames SILVER Fraudulent Online 22039.56 Abbottmunoz Travel India Ahmedabad Gujarat India 2023 true null T1003 C1003 Card1003 M1003 L1002 2025-03-09 1947-06-06 2029-05-19 Valeriegray MALE valerie.gray@yahoo.com 8745391478 Lucknow Debit Carterfullerandmcclure GOLD sucsess atm 16082.2 Raybush Retail Inda Mumbai Maharashtra India 2025 true null T1004 C1004 Card1004 M1004 L1005 2023-07-27 1989-06-21 2027-05-31 Victoriawyatt OTHER victoria.wyatt@outlook.com 9077975168 Chandigarh Credit Jamesgroup GOLD Fraudulent POS 19971.22 Flowersmartinandkelly null India Kolkata West Bengal India 2023 true null T1005 C1005 Card1005 M1005 L1002 2024-11-23 1962-08-22 2026-01-29 Carolyndaniel FEMALE carolyn.daniel@gmail.com 9895169156 Lucknow Debit Fordllc PLATINUM Failed onlne 92898.46 Wattsrobinsonandnguyen Grocery India Mumbai Maharashtra India 2024 true null T1006 C1006 Card1006 M1006 L1010 2025-02-07 1997-04-12 2025-12-12 Andrewstewart FEMALE andrew.stewart@yahoo.com null Mumbai Credit Jonesgentry GOLD Success Online 5981.49 Walteredwardsandrios Grocery Inda Kochi Kerala India 2025 true null T1007 C1007 Card1007 M1007 L1009 2024-04-24 2004-09-03 2029-05-17 Charlesmcgee OTHER charles.mcgee@rediffmail.com 6634806607 Ahmedabad Credit Bakerrichardsandhurst GOLD Failed POS 11015.98 Coxosborn Retail India Chandigarh Punjab India 2024 true null T1008 C1008 Card1008 M1008 L1006 2023-10-07 1979-02-23 2029-09-02 Anthonyrodriguez OTHER anthony.rodriguez@gmail.com 7142403927 Ahmedabad Credit Moorebeckerandcarlson SILVER FAIL onlne 42358.45 Jamesferrell Retail Inda Hyderabad Telangana India 2023 true null T1009 C1009 Card1009 M1009 L1005 2024-08-05 1961-07-31 2030-03-06 Alexandrale FEMALE alexandra.le@yahoo.com 6800849550 Kolkata Credit Dyerpotterandmack GOLD FAIL atm 65771.58 Rodriguezgraham Electronics India Kolkata West Bengal India 2024 true null T1010 C1010 Card1010 M1010 L1002 2024-06-08 1944-09-10 2027-09-18 Pamelaromero OTHER pamela.romero@gmail.com 6022988318 Chandigarh Debit Harrellllc SILVER FAIL Online 61206.61 Romerogonzalezandbrooks Grocery India Mumbai Maharashtra India 2024 true null T1011 C1011 Card1011 M1011 L1001 2023-10-25 1981-03-06 2029-12-25 Lindaburns OTHER linda.burns@yahoo.com 7685290680 Ahmedabad Debit Hickmanllc SILVER Fraudulent POS 13168.88 Brownplc Grocery Inda Bangalore Karnataka India 2023 true null T1012 C1012 Card1012 M1012 L1003 2023-08-22 1950-04-20 2029-08-09 Vanessapatel MALE vanessa.patel@gmail.com 6822178461 Chennai Debit Andersongroup GOLD Failed pos 86474.11 Martinroseandobrien Grocery India Chennai Tamil Nadu India 2023 true null T1013 C1013 Card1013 M1013 L1006 2023-12-03 1981-03-04 2027-03-08 Jenniferpowers MALE jennifer.powers@yahoo.com 9388263207 Mumbai Debit Kimholmes GOLD FAIL ATM 50989.63 Wrightinc Electronics India Hyderabad Telangana India 2023 true null T1014 C1014 Card1014 M1014 L1003 2024-03-10 1945-11-26 2030-03-08 Danieljones FEMALE daniel.jones@yahoo.com 9095776306 Chandigarh Debit Brownjonesandjohnson GOLD Success onlne 31460.17 Jonesyoung Travel 

- Normalize `Transaction_Status` values (case-insensitive):
  - "sucsess", "success" → "Success"
  - "fail", "failed" → "Failed"
  - "fraudulent" → "Fraudulent"
  - Others → "Unknown"

In [0]:
# Normalize Transaction_Type values (case-insensitive):
#  🔹 “atm” → “ATM”
#  🔹 “pos” → “POS”
#  🔹 “online”, “onlne” → “ONLINE”
#  🔹 Others → “UNKNOWN”


In [0]:
df = df.withColumn(
    "Transaction_Status",
    when(lower(trim(col("Transaction_Status"))).isin("sucsess", "success"),    lit("Success"))
    .when(lower(trim(col("Transaction_Status"))).isin("fail",  "faild"  "failed"),    lit("Failed"))
    .when(lower(trim(col("Transaction_Status")))== "fraudulent",               lit("Fraudulent"))
    .otherwise(lit("Unknown"))
)

df = df.withColumn(
    "Transaction_Type",
    when(lower(trim(col("Transaction_Type"))) == "atm",     lit("ATM"))
    .when(lower(trim(col("Transaction_Type"))) == "pos",     lit("POS"))
    .when(lower(trim(col("Transaction_Type"))).isin("online","onlne"), lit("ONLINE"))
    .otherwise(lit("UNKNOWN"))
)

In [0]:
# 4. Column order as per target table
target_cols = [
    "Transaction_ID", "Customer_ID", "Card_ID", "Merchant_ID", "Location_ID",
    "Transaction_Date", "DOB", "Expiry_Date",
    "Customer_Name", "Gender", "Email", "Phone", "Customer_City",
    "Card_Type", "Issuer_Bank", "Card_Tier",
     "Transaction_Status", "Transaction_Type", "Transaction_Amount",
    "Merchant_Name", "Merchant_Category", "Merchant_Country",
    "City", "State", "Country"
]
df_final = df.select([col(c) for c in target_cols])

In [0]:
# Write to Silver Delta table (overwrite)
df_final.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("prod.silver.clean_transactions")

In [0]:
%sql
select * from prod.silver.clean_transactions;

Transaction_ID Customer_ID Card_ID Merchant_ID Location_ID Transaction_Date DOB Expiry_Date Customer_Name Gender Email Phone Customer_City Card_Type Issuer_Bank Card_Tier Transaction_Status Transaction_Type Transaction_Amount Merchant_Name Merchant_Category Merchant_Country City State Country T1232 C1232 Card1232 M1232 L1232 2024-07-25 2005-09-23 2026-04-13 Pauljenkins MALE null 7166329471 Delhi Credit Riddleltd PLATINUM Success POS -100.0 Milesandsons Retail India Mumbai Westbengal India T1594 C1594 Card1594 M1594 L1594 2024-03-29 2005-12-07 2026-07-06 Cynthiaramos FEMALE cynthia.ramos@gmail.com 6951183270 Ahmedabad Credit Browninggroup GOLD Success ATM 39513.62 Turnermayer Grocery Inda Chennai Telangana India T1527 C1527 Card1527 M1527 L1527 2024-10-29 1986-02-01 2026-12-01 Kristenhall OTHER kristen.hall@yahoo.com 6030476354 Kochi Debit Vincentburns PLATINUM Failed ATM 96214.28 Vargasgroup Grocery Inda Ahmedabad Tamilnadu India T1276 C1276 Card1276 M1276 L1276 2024-09-08 1971-02-14 2030-04-20 Lancemccall FEMALE lance.mccall@outlook.com null Bangalore Credit Walkercunninghamandzuniga GOLD Success ATM 64679.68 Riverainc Electronics India Chandigarh Maharashtra India T1413 C1413 Card1413 M1413 L1413 2024-04-05 1996-12-04 2026-03-27 Jonathanparks OTHER jonathan.parks@gmail.com 9618873468 Bangalore Credit Galvanshea PLATINUM Fraudulent POS 19179.3 Brownkellyandmoore Travel India Mumbai Telangana India T1335 C1335 Card1335 M1335 L1335 2024-12-06 1967-12-21 2026-08-08 Misstinagordon FEMALE miss.tina.gordon@rediffmail.com 7741444815 Chennai Credit Walshpageandhodges PLATINUM Fraudulent ONLINE 18122.62 Castilloduncanandcontreras Retail India Mumbai Uttarpradesh India T1804 C1804 Card1804 M1804 L1804 2024-08-08 2003-09-11 2029-11-03 Carolchristensen FEMALE carol.christensen@outlook.com 7185923941 Lucknow Debit Hartmyers PLATINUM Success ONLINE 12804.59 Nelsonwhiteandmoore Electronics India Kochi Maharashtra India T1475 C1475 Card1475 M1475 L1475 2024-01-28 1983-07-16 2027-07-22 Ericmcclain OTHER eric.mcclain@yahoo.com 8998380126 Kolkata Credit Wallaceandersonandruiz PLATINUM Failed ONLINE 42341.78 Williamsreyesandmiller Travel India Bangalore Uttarpradesh India T1969 C1969 Card1969 M1969 L1969 2024-06-08 1987-05-22 2029-11-01 Angelawatts OTHER angela.watts@outlook.com 6162088177 Lucknow Debit Hamiltonltd GOLD Fraudulent ONLINE 22643.47 Stewartskinnerandguerrero Retail India Mumbai Delhi India T1929 C1929 Card1929 M1929 L1929 2024-01-10 1958-02-12 2028-04-26 Amyharris FEMALE amy.harris@rediffmail.com 7721702196 Kochi Credit Riveratoddandgaines GOLD Failed ONLINE 1510.15 Younginc Grocery Inda Kochi Telangana India T1686 C1686 Card1686 M1686 L1686 2024-12-10 1976-04-12 2028-04-15 Kylierogers OTHER kylie.rogers@gmail.com 7336080436 Lucknow Credit Barnesmartin GOLD Success ATM 3859.74 Walterlozano Travel India Kochi Uttarpradesh India T1392 C1392 Card1392 M1392 L1392 2024-09-11 1991-02-26 2027-06-02 Michellebrock MALE michelle.brock@gmail.com 8137346496 Mumbai Credit Reynoldsjamesandthomas PLATINUM Unknown ATM 4556.6 Hernandezshafferandthompson Travel India Lucknow Kerala India T1585 C1585 Card1585 M1585 L1585 2024-06-15 1954-07-12 2027-07-08 Michaelevans FEMALE michael.evans@yahoo.com 9611010485 Chennai Debit Jenkinsdalton GOLD Fraudulent ATM 62863.74 Waltersgroup Electronics India Lucknow Delhi India T1456 C1456 Card1456 M1456 L1456 2024-09-03 2003-10-07 2026-10-01 Charleshammond MALE charles.hammond@gmail.com 8134599918 Bangalore Credit Barnesllc GOLD Unknown ONLINE 51139.37 Mackgriffinandsims Grocery Inda Chandigarh Punjab India T1890 C1890 Card1890 M1890 L1890 2024-10-05 1978-01-23 2029-11-19 Michaellucas FEMALE michael.lucas@outlook.com 9076936610 Hyderabad Credit Robinsoninc PLATINUM Success ONLINE 49945.43 Johnstonandsons Electronics India Chennai Karnataka India T1384 C1384 Card1384 M1384 L1384 2024-06-30 1952-05-20 2027-09-01 Markgamble FEMALE mark.gamble@outlook.com 8782907163 Lucknow Credit Haynesmcbrideandgutierrez GOLD Succes